<a href="https://colab.research.google.com/github/123noob1/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep = ',', header = True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
df.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [7]:
df.describe()

DataFrame[summary: string, id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [92]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
#    Using SQL
print('Using spark SQL')
spark.sql('''
  SELECT date_built, ROUND(AVG(price), 2)
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY 1
  ORDER BY 1
''').show()

#    Using Spark DataFrame's GroupBy agg() function with round and avg
#    First import the required agg functions
from pyspark.sql.functions import avg, round

print('Using spark DataFrame\'s groupBy agg() function with round() and avg()')
price_avg_4br = df.filter('bedrooms = 4').sort('date_built')
price_avg_4br.groupBy(['date_built']).agg(round(avg('price'), 2)).show()

Using spark SQL
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           296800.75|
|      2011|            302141.9|
|      2012|           298233.42|
|      2013|           299999.39|
|      2014|           299073.89|
|      2015|           307908.86|
|      2016|           296050.24|
|      2017|           296576.69|
+----------+--------------------+

Using spark DataFrame's groupBy agg() function with round() and avg()
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           296800.75|
|      2011|            302141.9|
|      2012|           298233.42|
|      2013|           299999.39|
|      2014|           299073.89|
|      2015|           307908.86|
|      2016|           296050.24|
|      2017|           296576.69|
+----------+--------------------+



In [93]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
#    Using SQL
print('Using spark SQL')
spark.sql('''
  SELECT date_built, ROUND(AVG(price), 2)
  FROM home_sales
  WHERE bedrooms = 3 AND
    bathrooms = 3
  GROUP BY 1
  ORDER BY 1
''').show()

#    Using SPark DataFrame's GroupBy
print('Using spark DataFrame\'s groupBy agg() function with round() and avg()')
price_avg_3br_3bth = df.filter('bedrooms = 3 and bathrooms = 3').sort('date_built')
price_avg_3br_3bth.groupby(['date_built']).agg(round(avg('price'), 2)).show()

Using spark SQL
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+

Using spark DataFrame's groupBy agg() function with round() and avg()
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [94]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#    Using SQL
print('Using spark SQL')
spark.sql('''
  SELECT date_built, ROUND(AVG(price), 2)
  FROM home_sales
  WHERE bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
  GROUP BY 1
  ORDER BY 1
''').show()

#    Using SPark DataFrame's GroupBy
print('Using spark DataFrame\'s groupBy agg() function with round() and avg()')
price_avg_3br_3bth_2 = df.filter('bedrooms = 3 and bathrooms = 3 AND floors = 2 AND sqft_living >= 2000').sort('date_built')
price_avg_3br_3bth_2.groupby(['date_built']).agg(round(avg('price'), 2)).show()

Using spark SQL
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+

Using spark DataFrame's groupBy agg() function with round() and avg()
+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [112]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
#    id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view

start_time = time.time()

spark.sql('''
  SELECT INT(view) view, ROUND(AVG(price), 2)
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 1
''').show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.45329785346984863 seconds ---


In [113]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [114]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [115]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

spark.sql('''
  SELECT INT(view) view, ROUND(AVG(price), 2)
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 1
''').show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.3034942150115967 seconds ---


In [116]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_p')

In [117]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('home_sales_p')

In [118]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [119]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()

spark.sql('''
  SELECT INT(view) view, ROUND(AVG(price), 2)
  FROM p_home_sales_p
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 1
''').show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.4875931739807129 seconds ---


In [120]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [121]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False